In [17]:
import torch
import torch.nn as nn
from torchsummary import summary
import torch.nn.functional as F


In [32]:
# Functions to move data to GPU
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    else:
        return data.to(device, non_blocking=True)

# Arquitectura Xception

## Actividad 1
Investigue y explique la mejora que introduce la arquitectura Xception en relación a su predecesor Inception.
Como hint, estudie el concepto de Depthwise Separable Convolution y cómo impacta esto en el tiempo de entrenamiento de la red.

Xception introduce dos mejoras principales:

**Depthwise Separable Convolution**: es una alternativa a la convolución clásica para ser más eficiente en términos de tiempo de computación. Se divide en dos pasos:
* Depthwise convolution: es un primer paso donde en vez de aplicar una convolución $d \times d \times C$ donde $d$ es el tamaño del kernel y $C$ el número de canales, aplica una convolución $d \times d \times 1$ es decir a un solo canal. 
* Pointwise convolution: opera una convolución clásica de tamaño $1 \times 1 \times N$ donde $N$ es el número de kernels.
Siguiendo este procedimiento se reduce el número de operaciones en un factor propocional a $1/N$.

**Shortcuts entre bloques de convolución como en ResNet**: implementa el uso de bloques residuales, es decir que cada capa alimenta a la siguiente capa y también de manera directa saltando algunas capas intermedias. La idea detrás es evitar el "vanishing gradient" y poder entrenar redes más profundas.

## Actividad 2
Explique, en no más de tres lı́neas cada uno, la utilidad o función que cumplen en la arquitectura de una CNNs:

**1. Capa densa (Fully connected layer)**
Las capas densas usualmente son ubicadas a la salida de la red con fines de clasificación. Usualmente tiene el mismo número de nodos de salida que el número de clases y junto a una función softmax lo cual nos permite interpretar los valores de esta última capa como la probabilidad de pertenecer a una cierta clase.

**2. Kernel de convolución 1x1**
El uso principal es el cambio de dimensionalidad en el espacio del filtro. Si el número de filtros convolucionales de salida es mayor que el de entrada $F_1 > F$ se incrementa la dimensionalidad, caso contrario $F_1 < F$, se reduce. Reducir la dimensionalidad reduce el costo computacional. 

**3. MaxPooling**
MaxPooling es usado para reducir la "resolución" de una capa convolucional, la red estará "mirando" áreas más grandes de la imagen y reducirá el número de parámetros, por lo tanto reducirá el costo computacional. También ayudará a "ver" los pixeles más activados o importantes descartando los otros.



## Actividad 3
Como primera actividad, tendrán que recrear la operación de Depthwise Separable Convolution rellenando el siguiente código:

In [18]:
class SeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=False, **kwargs):
        super(SeparableConv2d, self).__init__()
        
        # aplicamos una convolución por canal 
        self.depthwise_conv = nn.Conv2d(
            in_channels, in_channels, groups=in_channels, kernel_size=kernel_size, padding='valid', bias=bias)
        self.pointwise_conv = nn.Conv2d(
            in_channels, out_channels, kernel_size=1, padding='valid', bias=bias)

    def forward(self, x):
        # No cambiar
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)

        return x


Además, responda explı́citamente: ¿cuáles son los valores de los parámetros kernel_size y stride en
la operación self.pointwise_conv?.

* kernel_size = 1 (filtros de 1 x 1)
* stride = 1 (por defecto)

## Actividad 4
Compare el número de parámetros entre un bloque de SeparableConv2d y una convolución para un vol-
umen de entrada de forma [3, 300, 300].

Considerando un out_channel = 10, tenemos:
* SeparableConv2d: 57 parámetros
* Classic Conv2d: 270 parámetros

In [19]:
model_sep = nn.Sequential(SeparableConv2d(3,10,3))
summary(model_sep.cuda(), input_size=(3,300,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 298, 298]              27
            Conv2d-2         [-1, 10, 298, 298]              30
   SeparableConv2d-3         [-1, 10, 298, 298]               0
Total params: 57
Trainable params: 57
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.03
Forward/backward pass size (MB): 15.58
Params size (MB): 0.00
Estimated Total Size (MB): 16.61
----------------------------------------------------------------


In [20]:
model_classic = nn.Sequential(
    nn.Conv2d(3, 10, kernel_size=3, padding='valid', bias=False))
summary(model_classic.cuda(), input_size=(3,300,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 10, 298, 298]             270
Total params: 270
Trainable params: 270
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.03
Forward/backward pass size (MB): 6.78
Params size (MB): 0.00
Estimated Total Size (MB): 7.81
----------------------------------------------------------------


## Actividad 5
En esta segunda actividad crearemos el bloque base de la red, que se compone de etapas subsecuentes de la operación SeparableConv2d además de ReLU, BatchNorm y MaxPooling, junto con una conexión
skip-forward que varı́a según la etapa donde nos encontramos. Para esto, tendrá que completar la siguiente clase:

In [84]:
class XceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, start_with_relu=True,
                 residual_connection=False, num_subblocks=2,
                 last_layer='MaxPooling', **kwargs):
        super(XceptionBlock, self).__init__()

        modules = []
        # # Agregue los módulos según corresponda
        # # ...

        # self.modules = nn.Sequential(*modules)

        # Agregamos sub-bloques segun corresponda
        for _ in range(num_subblocks):
            modules += self._subblock(out_channels, out_channels)

        # Agregamos layer final segun corresponda
        if last_layer == 'MaxPooling':
            modules.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

        # Agregamos conexion skip-forward
        if residual_connection:
            self.skip_forward = None
        else:
            self.skip_forward = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )

        # Definimos si partimos o no con ReLU
        if not start_with_relu:
            modules = modules[1:]

        self.modules = nn.Sequential(*modules)

    def _subblock(self, in_channels, out_channels):
        """
        Construye un sub-bloque compuesto de:
            ReLU -> SeparableConv2d -> BatchNormalization 

        Admite no incluir la ReLU mediante el parametro "start_with_relu"
        """
        modules = []
        modules.append(nn.ReLU(inplace=True))
        modules.append(SeparableConv2d(
            in_channels, out_channels, kernel_size=3, stride=1))
        modules.append(nn.BatchNorm2d(out_channels))

        return modules

    def forward(self, x):
        """
        Forward pass de un bloque.
        """
        # No cambiar
        Fx = self.modules(x)

        if not self.skip_forward:
            Fx += x
        else:
            Fx += self.skip_forward(x)

        return Fx


## Actividad 6
Por último, complete la clase Xception que instancia el modelo completo. Para esto, tendrá que usar la clase XceptionBlock, instanciando cada etapa por separado en la inicialización de la clase y rellenando las funciones para cada flujo. Note que el método Xception.forward ya está implementado.

In [89]:
class Xception(nn.Module):
    def __init__(self, num_classes):

        super(Xception, self).__init__()
        self.num_classes = num_classes

        # Head de clasificacion para ilustrar
        self.fc = nn.Linear(2048, num_classes)
        # Aqui debe agregar todos los bloques y etapas
        # que necesitara para las funciones de cada flujo.
        # ...

    def entry_flow(self, x):
        """
        Flujo de entrada (debera usar la funcion
        XceptionBlock tres veces mas las dos capas
        convolucionales vistas en el enunciado)
        """
        # Agregar flujo de entrada
        # ...
        # Primeros dos bloques
        first_block = [
            nn.Conv2d(3, 32, kernel_size=3, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(inplace=True)
        ]
        second_block = [
            XceptionBlock(64, 128, start_with_relu=False,
                          residual_connection=False, last_layer=None),
            XceptionBlock(64, 128, start_with_relu=True,
                          residual_connection=True, last_layer='MaxPooling'),
        ]
        third_block = [
            XceptionBlock(128, 256, start_with_relu=True,
                          residual_connection=False, last_layer=False),
            XceptionBlock(128, 256, start_with_relu=True,
                          residual_connection=True, last_layer='MaxPooling')
        ]
        fourth_block = [
            XceptionBlock(256, 728, start_with_relu=True,
                          residual_connection=False, last_layer=None),
            XceptionBlock(256, 728, start_with_relu=True,
                          residual_connection=True, last_layer='MaxPooling')
        ]

        # Agregue los módulos según corresponda
        # ...
        modules = nn.Sequential(
            *first_block, *second_block, *third_block, *fourth_block).to(get_default_device())

        return modules

    def middle_flow(self, x):
        """
        Flujo intermedio (recordar que se repite 8 veces
        un XceptionBlock de determinados parametros)
        """
        # Agregar flujo intermedio
        # ...
        block = [
            XceptionBlock(728, 728, start_with_relu=True,
                          residual_connection=False, last_layer=None),
            XceptionBlock(728, 728, start_with_relu=True,
                          residual_connection=False, last_layer=None),
            XceptionBlock(728, 728, start_with_relu=True,
                          residual_connection=True, last_layer=None),
        ]

        full_block = []
        full_block.extend([block] * 8)

        modules = nn.Sequential(*full_block).to(get_default_device())

        return modules

    def exit_flow(self, x):
        """
        Flujo de salida (tener en cuenta cambio de
        dimensiones entre sub-bloques y funcion de
        salida)
        """
        # Agregar flujo de salida
        # ...
        first_block = [
            XceptionBlock(728, 728, start_with_relu=True,
                          residual_connection=False, last_layer=None),
            XceptionBlock(728, 1024, start_with_relu=True,
                          residual_connection=True, last_layer='MaxPooling')
        ]
        second_block = [
            XceptionBlock(1024, 1536, start_with_relu=False,
                          residual_connection=False, last_layer=None),
            nn.ReLU(inplace=True),
            XceptionBlock(1536, 2048, start_with_relu=False,
                          residual_connection=False, last_layer=None),
            nn.ReLU(inplace=True),
        ]

        modules = nn.Sequential(
            *first_block, *second_block).to(get_default_device())

        # No olvidar etapa de Global Avg. Pooling
        # Hint: https://discuss.pytorch.org/t/global-average-pooling-in-pytorch/6721/8
        # x = GlobalAvgPooling(x)
        modules.append(F.adaptive_avg_pool2d(x, (1,1)))

        return x

    def forward(self, x):
        x = self.entry_flow(x)
        x = self.middle_flow(x)
        x = self.exit_flow(x)
        # Hay que agregar un cambio de dimensiones para poder
        # pasar x por la capa Fully Connected
        # x = ...
        x = nn.Flatten(-1, 2048)
        x = self.fc(x)
        return x


## Actividad 7
Usando la función torchsummary.summary(), visualice y verifique que los tamaños de los outputs de
cada flujo sean los correctos. Explicite el nombre exacto de cada capa y sus dimensiones.

In [23]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# inputs, labels = inputs.to(device), labels.to(device)


In [90]:
model_xception = Xception(2048)
summary(model_xception.to(get_default_device()), input_size=(3,300,300))

TypeError: list is not a Module subclass

# Implementación y entrenamiento de un clasificador de perros

In [ ]:
device = torch.device("cuda")
# model.cuda()